# Phase 1 - Récuperer les données

In [1]:
train_url = 'http://bit.ly/titanic-train-set'
test_url  = 'http://bit.ly/titanic-test-set'

In [2]:
import pandas as pd

# Si il ya un split train / test, on le fait.

# Train Set
df = pd.read_csv(train_url, index_col="PassengerId")
# Test Set
df_test = pd.read_csv(test_url, index_col="PassengerId")

df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
df_test.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


# Phase 2 - Isoler les features `X` des cibles `Y`

In [ ]:
# Pour le train test
colonne_cible = "Survived"

X = df.drop(colonne_cible, axis='columns')
Y = df[colonne_cible]

In [ ]:
# Pour le test
X_test = df_test

# Phase 3 - EDA

On va juste regarder les valeurs nulles.

(en théorie il faudrait aussi recharcher la présence de valeur aberrante, tracer des graphiques etc...)

In [ ]:
# Check valeur nulle sur le train
X.isna().sum()

Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [ ]:
# Check valeur nulle sur le test
X_test.isna().sum()

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
dtype: int64

# Phase 4 - Déterminer quoi faire avec chaque colonne

In [ ]:
# Les colonnes sur lesquelles on va faire un traitement
colonnes_catégoriques = ['Sex']
colonnes_numériques = []
colonnes_catégoriques_avec_valeurs_manquantes = ['Embarked']
colonnes_numériques_avec_valeurs_manquantes = ['Age', 'Fare']
colonnes_feature_engineering = ['Name' , 'Cabin']  # CountVectorizer / Cabin etc...

# Le reste
drop_colonnes = ['Ticket']  # Colonnes à lacher
passthrough_colonnes = ['Pclass', 'SibSp', 'Parch'] # Colonnes à laisser-passer telles quelles

In [ ]:
# Vérifier qu'on a tout

all_columns = (colonnes_catégoriques 
               + colonnes_numériques 
               + colonnes_catégoriques_avec_valeurs_manquantes
               + colonnes_numériques_avec_valeurs_manquantes
               + colonnes_feature_engineering
               + drop_colonnes
               + passthrough_colonnes)

if set(all_columns) == set(X.columns):
  print('Ok')
else:
  # Alors je veux voir les différences
  print('dans all_columns mais pas dans X.columns   :', set(all_columns) - set(X.columns))
  print('dans X.columns   mais pas dans all_columns :', set(X.columns) - set(all_columns))

Ok


# Phase 5 - Data Cleaning

### 1. Importer les outils 

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import FunctionTransformer

from sklearn.

### 2. Créer les transformers complexes (ceux qui sont pas ok, out-of-box)

Dans notre cas:


*   `'Embarked'` : Valeur Manquante -> Categorique [One Hot] 
*   `'Cabin'` : Feature Engineering -> Valeur Manquante -> Categorique [One Hot] 

https://towardsdatascience.com/pipeline-columntransformer-and-featureunion-explained-f5491f815f

In [ ]:
fill_missing_then_one_hot_encoder = make_pipeline(
    SimpleImputer(strategy='constant', fill_value='manquante'),
    OneHotEncoder(handle_unknown='ignore')
)

In [ ]:
def extraire_la_première_lettre(serie):
    return pd.DataFrame(serie.str[0])

extraire_lettre_cabine = make_pipeline(
    FunctionTransformer(extraire_la_première_lettre),
    fill_missing_then_one_hot_encoder,
)

### 3. Les combiner dans un ColumnTransformer

In [ ]:
data_cleaning = make_column_transformer(
    ( OneHotEncoder(), colonnes_catégoriques ),
    ( fill_missing_then_one_hot_encoder , colonnes_catégoriques_avec_valeurs_manquantes),
    ( SimpleImputer(strategy='mean'), colonnes_numériques_avec_valeurs_manquantes),
    ( CountVectorizer(), 'Name'),
    ( extraire_lettre_cabine, 'Cabin'),
    ( 'drop' , drop_colonnes),
    ( 'passthrough' , passthrough_colonnes)
)


In [ ]:
# Il est conseillé de vérfiier que ça marche
# pour ça, on va voir si on peut fit transform sans erreur

# data_cleaning.fit(X)

# data_cleaning.transform(X)

# data_cleaning.transform(X_test)

print('C\'est bon')

C'est bon


# Phase 6 - Trouver la Meilleure Pipeline



In [ ]:
from sklearn.model_selection import GridSearchCV

### étape 1 : Définir Procédure de Cross-Validation

In [1]:
from sklearn.model_selection import KFold

cross_validation_design = KFold(n_splits=5,
                                shuffle=True,
                                random_state=77)

cross_validation_design

KFold(n_splits=5, random_state=77, shuffle=True)

### Pipeline 1 : KNN (pas de PCA)

In [ ]:
data_cleaning = make_column_transformer(
    ( OneHotEncoder(), colonnes_catégoriques ),
    ( fill_missing_then_one_hot_encoder , colonnes_catégoriques_avec_valeurs_manquantes),
    ( SimpleImputer(strategy='mean'), colonnes_numériques_avec_valeurs_manquantes),
    ( CountVectorizer(), 'Name'),
    ( extraire_lettre_cabine, 'Cabin'),
    ( 'drop' , drop_colonnes),
    ( 'passthrough' , passthrough_colonnes)
)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier as KNN

KNN_MODEL = {}

# Définir la pipeline
KNN_MODEL['pipeline'] = Pipeline([
                                  ('data_cleaning', data_cleaning),
                                  ('knn', KNN())
                                  ])

# Définir la grille
KNN_MODEL['hyperparams'] = {}
KNN_MODEL['hyperparams']['knn__n_neighbors'] = [1, 3, 9, 21, 51]
KNN_MODEL['hyperparams']['knn__weights'] = ['uniform', 'distance']

# Effectuer la GridSearch
KNN_MODEL['gridsearch'] = GridSearchCV(
    estimator=KNN_MODEL['pipeline'],
    param_grid=KNN_MODEL['hyperparams'],
    cv=cross_validation_design,
    scoring='accuracy'
    )

KNN_MODEL['gridsearch'].fit(X, Y)

GridSearchCV(cv=KFold(n_splits=5, random_state=77, shuffle=True),
             error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('data_cleaning',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('onehotencoder',
                                                                         OneHotEncoder(categories='auto',
                                                                                       drop=None,
                                                                                       dtype=<class 'numpy.float64'>,
                                                                               

In [ ]:
KNN_MODEL['gridsearch'].best_params_

{'knn__n_neighbors': 9, 'knn__weights': 'distance'}

In [ ]:
KNN_MODEL['gridsearch'].best_score_

0.7205511267340405

### Pipeline 2 : RidgeRegression Classifier (un type de régression linéaire avec Feature Selection intégrée)

In [ ]:
data_cleaning = make_column_transformer(
    ( OneHotEncoder(), colonnes_catégoriques ),
    ( fill_missing_then_one_hot_encoder , colonnes_catégoriques_avec_valeurs_manquantes),
    ( SimpleImputer(strategy='mean'), colonnes_numériques_avec_valeurs_manquantes),
    ( CountVectorizer(), 'Name'),
    ( extraire_lettre_cabine, 'Cabin'),
    ( 'drop' , drop_colonnes),
    ( 'passthrough' , passthrough_colonnes)
)

In [ ]:
from sklearn.linear_model import RidgeClassifier

# Définir la pipeline
REGRESSION_MODEL = {}
REGRESSION_MODEL['pipeline'] = Pipeline([
                                        ('data_cleaning', data_cleaning ),
                                        ('reg', RidgeClassifier())
])

# Définir la grille
REGRESSION_MODEL['hyperparams'] = {}
REGRESSION_MODEL['hyperparams']['reg__alpha'] = np.arange(.1, 10., .1)

# Faire la recherche
REGRESSION_MODEL['gridsearch'] = GridSearchCV(
                                      estimator=REGRESSION_MODEL['pipeline'],
                                      param_grid=REGRESSION_MODEL['hyperparams'],
                                      cv=cross_validation_design,
                                      scoring='accuracy'
                                      )

REGRESSION_MODEL['gridsearch'].fit(X, Y)

GridSearchCV(cv=KFold(n_splits=5, random_state=77, shuffle=True),
             error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('data_cleaning',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('onehotencoder',
                                                                         OneHotEncoder(categories='auto',
                                                                                       drop=None,
                                                                                       dtype=<class 'numpy.float64'>,
                                                                               

In [ ]:
REGRESSION_MODEL['gridsearch'].best_params_

{'reg__alpha': 5.1}

In [ ]:
REGRESSION_MODEL['gridsearch'].best_score_

0.830500282468144

### Pipeline 3 - RandomForest (le meilleur modèle quand on y connait rien)

In [ ]:
data_cleaning = make_column_transformer(
    ( OneHotEncoder(), colonnes_catégoriques ),
    ( fill_missing_then_one_hot_encoder , colonnes_catégoriques_avec_valeurs_manquantes),
    ( SimpleImputer(strategy='mean'), colonnes_numériques_avec_valeurs_manquantes),
    ( CountVectorizer(), 'Name'),
    ( extraire_lettre_cabine, 'Cabin'),
    ( 'drop' , drop_colonnes),
    ( 'passthrough' , passthrough_colonnes)
)

In [ ]:
from sklearn.ensemble import RandomForestClassifier as RandomForest

RF_MODEL = {}

# Définir la pipe
RF_MODEL['pipeline']=Pipeline([
                               ('data_cleaning', data_cleaning),
                               ('rf', RandomForest(random_state=7))])

# Définir la grille d'hyperparams
RF_MODEL['hyperparams'] = {}
RF_MODEL['hyperparams']['rf__n_estimators'] = [10, 30, 50, 100, 150]
RF_MODEL['hyperparams']['rf__max_features'] = [None, 'sqrt', 'log2', .1, .25, .50, .75, .85]
RF_MODEL['hyperparams']['rf__max_depth'] = [None, 4, 7, 10, 20]

# Recherche d'HP
RF_MODEL['gridsearch'] = GridSearchCV(
    estimator=RF_MODEL['pipeline'],
    param_grid=RF_MODEL['hyperparams'],
    scoring='accuracy',
    cv=cross_validation_design
)

RF_MODEL['gridsearch'].fit(X, Y)

GridSearchCV(cv=KFold(n_splits=5, random_state=77, shuffle=True),
             error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('data_cleaning',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('onehotencoder',
                                                                         OneHotEncoder(categories='auto',
                                                                                       drop=None,
                                                                                       dtype=<class 'numpy.float64'>,
                                                                               

In [ ]:
RF_MODEL['gridsearch'].best_params_

{'rf__max_depth': 10, 'rf__max_features': 0.25, 'rf__n_estimators': 150}

In [ ]:
RF_MODEL['gridsearch'].best_score_

0.8350134957002071

# On selectionne le meilleur modèle : on va réentrainer sur tout (sans cross val) avec les meilleurs params

In [ ]:
data_cleaning = make_column_transformer(
    ( OneHotEncoder(), colonnes_catégoriques ),
    ( fill_missing_then_one_hot_encoder , colonnes_catégoriques_avec_valeurs_manquantes),
    ( SimpleImputer(strategy='mean'), colonnes_numériques_avec_valeurs_manquantes),
    ( CountVectorizer(), 'Name'),
    ( extraire_lettre_cabine, 'Cabin'),
    ( 'drop' , drop_colonnes),
    ( 'passthrough' , passthrough_colonnes)
)

model_final = Pipeline([('data_cleaning', data_cleaning),
                        ('rf', RandomForest(random_state=7,
                                            max_depth=10,
                                            max_features=0.25,
                                            n_estimators=150))
                        ])


# on fit la meilleur pipe sur toute nos données de train
model_final.fit(X, Y)

Pipeline(memory=None,
         steps=[('data_cleaning',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories='auto',
                                                                drop=None,
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='error',
                                                                sparse=True),
                                                  ['Sex']),
                                                 ('pipeline-1',
                                                  Pipeline(memory=None,
                                                           steps=[('simpleimpu

# Fini